<a href="https://colab.research.google.com/github/Marcosuff/qsar_tutorials/blob/master/nbs/Bioaffinity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import modules**

In [ ]:
%matplotlib.inline
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, 

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

# **Import data**

In [4]:
data = pd.read_csv('Sample data.csv')
data.head()

,Smiles,pXC50,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,Nc1nc(N)c2c(Sc3ccccc3)cccc2n1,6.208,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,COc1ccc(OC)c(Cc2sc3nc(N)nc(N)c3c2C)c1,6.137,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,CN(Cc1coc2nc(N)nc(N)c12)c3ccc(cc3)C(=O)N[C@@H]...,6.658,1,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0
3,Nc1nc(N)c2nc(CSc3ccc(cc3)C(=O)NC(CCC(=O)O)C(=O...,5.569,1,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0
4,Nc1nc(N)c2nc(CCSc3ccc(cc3)C(=O)NC(CCC(=O)O)C(=...,4.602,1,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0


In [12]:
print(data.shape)

(777, 1026)


In [13]:
cols = data.columns[2:]
len(cols)

1024

# Split data

In [21]:
int(777 * 0.1)

77

In [22]:
train,test = train_test_split(data,test_size=0.1,random_state=42)

In [23]:
x_train = train[cols].values
y_train = train['pXC50'].values

x_test = train[cols].values
y_tst = train['pXC50'].values

# Crossvalidation

**Define estimators**

In [51]:
# Models
linear = LinearRegression(n_jobs=-1)
ridge = Ridge()
rf = RandomForestRegressor()

**CV iterator and parameters to explore**

In [71]:
cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=42)

In [81]:
rf_params = {'n_estimators':np.arange(100, 3000, 200),
             'min_samples_split':np.arange(0.1,1.,0.1),
             'min_samples_leaf':np.arange(1,20,2)}
ridge_params = {'alpha':np.logspace(-3,1,20)}
linear_params = {'fit_intercept':[True, False]}
estimators = [(rf,rf_params),(ridge,ridge_params),(linear,linear_params)]

**Metrics**

In [ ]:
def _pearson_corref(ytrue,ypred):
  '''Calculate Pearson's correlation coefficient between ytrue and ypred
  ytrue:: true labels
  ypred:: predictions'''

  return pearsonr(ytrue,ypred)[0]


scoring = {'mse':'neg_mean_squared_error',
           'rmse':make_scorer(mean_squared_error,squared=False),
           'r2':'r2',
           'pearson_corr':make_scorer(_pearson_corref)}

**Fit GridSearch**

In [ ]:
results = []
grid = GridSearchCV(rf,rf_params,scoring=scoring,refit='rmse',cv=cv)
grid.fit(x_train,y_train)

In [94]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fit_intercept,params,split0_test_mse,split1_test_mse,split2_test_mse,split3_test_mse,split4_test_mse,mean_test_mse,std_test_mse,rank_test_mse,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_test_pearson_corr,split1_test_pearson_corr,split2_test_pearson_corr,split3_test_pearson_corr,split4_test_pearson_corr,mean_test_pearson_corr,std_test_pearson_corr,rank_test_pearson_corr
0,0.236504,0.022060,0.002535,0.000275,True,{'fit_intercept': True},-1.502166e+22,-1.524861e+21,-1.348732e+21,-1.352146e+22,-1.050770e+24,-2.164373e+23,4.172059e+23,2,1.225629e+11,3.904947e+10,3.672509e+10,1.162818e+11,1.025071e+12,2.679380e+11,3.803242e+11,1,-8.742172e+21,-7.686902e+20,-7.198410e+20,-7.783622e+21,-6.288663e+23,-1.293761e+23,2.497679e+23,2,-0.086187,-0.160839,0.014687,-0.002262,-0.063187,-0.059558,0.062889,2
1,0.225298,0.022098,0.003081,0.001106,False,{'fit_intercept': False},-8.379325e+21,-2.966471e+21,-3.169039e+19,-1.541331e+22,-5.727555e+20,-5.472710e+21,5.782030e+21,1,9.153865e+10,5.446532e+10,5.629422e+09,1.241503e+11,2.393231e+10,5.994321e+10,4.335345e+10,2,-4.876524e+21,-1.495413e+21,-1.691370e+19,-8.872666e+21,-3.427836e+20,-3.120860e+21,3.351415e+21,1,-0.105362,-0.014066,-0.010093,0.032195,-0.064014,-0.032268,0.047601,1


In [ ]:
#@title
r2 = r2_score(y_casf, preds)

#adj_r2 = 1-(1-r2)*(X_casf.shape[0]-1)/(X_casf.shape[0]-X_casf.shape[1]-1)
external_rp = (list(pearsonr(y_casf, preds)))[0]
external_sp = (list(spearmanr(y_casf, preds)))[0]

testmse=mean_squared_error(y_casf,preds)
num=X_casf.shape[0]
testsd=sqrt((testmse*num)/(num-1))

# Best fit line
a, b = best_fit(y_casf, preds)
yfit = [a + b * xi for xi in y_casf]


results_external = pd.DataFrame({'true_affi' : y_casf, 'preds' : preds})
#results_external.to_csv('fp+plants+rfscore_models/{}_test.csv'.format('RF_regressor_newrfscoreplants'),index=False)

fig, ax = plt.subplots()
#plt.title('CASF-2013 195 complexes\n kNN')
plt.title('CASF-2013 195 complexos\n Linear Regression')
fig.figsize=(7.5,5.5)
plt.scatter(x=y_casf, y=preds, edgecolors=(0, 0, 0),c='w')
plt.plot(y_casf, yfit,c='k')
anchored_text = AnchoredText('Rp = {:.3f}\nRs = {:.3f}'.format(external_rp,external_sp), loc=2)
ax.add_artist(anchored_text)

#plt.text(5,10,'$R^2$ = {:.3f}\n$AdjR^2$ = {:.3f}'.format(r2,adj_r2)) 
#plt.xlabel('Experimental binding constant (logKa)',size=12)
#plt.ylabel('Calculated binding constant (clogKa)',size=12)
plt.xlabel('Afinidade de ligação experimental (logKa)',size=12)
plt.ylabel('Afinidade de ligação calculada (clogKa)',size=12)
plt.xticks(size=10)
plt.yticks(size=10)
plt.savefig('Linear_regressor_pt.jpg',dpi=300)
plt.show()